In [2]:
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import numpy as np
import os
from urllib.parse import quote
from selenium import webdriver

In [12]:
csv_file = 'E:\Major Project 2\IPL Player Performance Prediction\Players data\scrap_mi2_file_2015to24.csv'
   
# Read the CSV file into a DataFrame
final_data = pd.read_csv(csv_file)
# print(final_data)
# List of columns to drop
columns_to_drop = ['50', '100', '4s Scored', '6s Scored']

# Drop the columns
final_data = final_data.drop(columns=columns_to_drop)
final_data

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,Bat Dot%,Overs Bowled,Runs Given,Wickets Taken,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,SA Yadav,ipl2015_01,46,20,0,230.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11,9,0.0,0.0,0.0
1,SA Yadav,ipl2015_05,11,9,1,122.2,44.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2,3,0.0,0.0,0.0
2,SA Yadav,ipl2015_10,2,3,1,66.7,33.3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23,10,0.0,0.0,0.0
3,SA Yadav,ipl2015_15,23,10,1,230.0,40.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,24,16,0.0,0.0,0.0
4,SA Yadav,ipl2015_18,24,16,1,150.0,25.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2,2,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,Vishnu Vinod,smat2021_050,71,37,0,191.9,13.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,10,0.0,0.0,0.0
1414,Vishnu Vinod,smat2021_088,10,10,1,100.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,65,26,0.0,0.0,0.0
1415,Vishnu Vinod,smat2021_2_099,65,26,0,250.0,19.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12,7,0.0,0.0,0.0
1416,Vishnu Vinod,smat2022_127,12,7,0,171.4,14.3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17,12,0.0,0.0,0.0


In [16]:
import pandas as pd
import numpy as np
import math
import time
from statistics import stdev
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

import scipy.stats

models = pd.DataFrame()
players_list= ['SA Yadav', 'RG Sharma', 'NT Tilak Varma', 'TM Head', 'D Brevis', 'JJ Bumrah', 'G Coetzee', 'PP Chawla', 'N Thushara', 'L Wood', 'HH Pandya', 'Mohammad Nabi', 'R Shepherd', 'Ishan Kishan', 'N Wadhera', 'Shivam Sharma', 'Naman Dhir', 'A Madhwal', 'S Gopal', 'K Kartikeya', 'AS Tendulkar', 'A Kamboj', 'SZ Mulani', 'Vishnu Vinod'] #[MI]
# players_list = ['DA Warner', 'PP Shaw', 'RK Bhui', 'YV Dhull', 'Kuldeep Yadav', 'Abishek Porel', 'RR Pant', 'SD Hope', 'K Kushagra', 'T Stubbs', 'A Nortje', 'I Sharma', 'Mukesh Kumar', 'P Dubey', 'KK Ahmed', 'Rasikh Salam', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh', 'Sumit Kumar', 'J Fraser-McGurk'] # [DC]
# players_list = ['D Padikkal', 'KR Mayers', 'DJ Hooda', 'AJ Turner', 'A Badoni', 'Ravi Bishnoi', 'Naveen-ul-Haq', 'S Joseph', 'Shivam Mavi', 'A Mishra', 'Mohsin Khan', 'Yash Thakur', 'Arshad Khan', 'M Siddharth', 'MP Yadav', 'KH Pandya', 'MP Stoinis', 'DJ Willey', 'K Gowtham Thamarai', 'PN Mankad', 'KL Rahul', 'N Pooran', 'Q de Kock'] # [LSG]
# players_list =  ['Shubman Gill', 'KS Williamson', 'B Sai Sudharsan', 'DA Miller', 'V Shankar', 'A Manohar', 'Rashid Khan', 'MM Sharma', 'JB Little', 'UT Yadav', 'Noor Ahmad', 'SH Johnson', 'Kartik Tyagi', 'R Sai Kishore', 'J Yadav', 'S Sandeep Warrier', 'DG Nalkande', 'Azmatullah Omarzai', 'R Tewatia', 'M Shahrukh Khan', 'MJ Suthar', 'WP Saha', 'MS Wade', 'BR Sharath'] #[GT]
# players_list = ['YBK Jaiswal', 'SO Hetmyer', 'R Powell', 'T Kohler-Cadmore', 'SB Dubey', 'TA Boult', 'YS Chahal', 'Avesh Khan', 'Sandeep Sharma', 'N Burger', 'NA Saini', 'KR Sen', 'Abid Mushtaq', 'R Ashwin', 'R Parag', 'D Ferreira', 'JC Buttler', 'SV Samson', 'DC Jurel', 'KS Rathore', 'TK Kotian'] #[RR]
# players_list = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]
# players_list =  ['RK Singh', 'SS Iyer', 'VR Iyer', 'N Rana', 'SE Rutherford', 'MK Pandey', 'A Raghuvanshi', 'MA Starc', 'CV Varun', 'PVD Chameera', 'Suyash Sharma', 'Mujeeb Ur Rahman', 'VG Arora', 'C Sakariya', 'Harshit Rana', 'AD Russell', 'SP Narine', 'Ramandeep Singh', 'Rahmanullah Gurbaz', 'PD Salt', 'KS Bharat', 'AS Roy'] #[KKR]
# players_list = ['V Kohli', 'F du Plessis', 'RM Patidar', 'Mahipal Lomaror', 'SS Prabhudessadi', 'SD Chauhan', 'Mohammed Siraj', 'LH Ferguson', 'RJW Topley', 'KV Sharma', 'AS Joseph', 'MJ Dagar', 'Yash Dayal', 'H Sharma', 'Rajan Kumar', 'GJ Maxwell', 'Cameron Green', 'TK Curran', 'WG Jacks',  'Swapnil Singh', 'MS Bhandage', 'KD Karthik', 'Anuj Rawat', 'V Vyshak'] #[RCB]
# players_list = ["TM Head", "MA Agarwal", "RA Tripathi", "GD Phillips", "Abhishek Sharma", "Abdul Samad", "PJ Cummins", "B Kumar", "T Natarajan", "M Markande", "Fazalhaq Farooqi", "JD Unadkat", "Akash Singh", "J Subramanyan", "AK Markram", "M Jansen", "PW Hasaranga", "Washington Sundar", "Shahbaz Ahmed", "K Nitish Kumar Reddy", "Sanvir Singh", "H Klaasen", "Anmolpreet Singh", "UD Yadav"] #[SRH]
# players_list = ['RD Gaikwad', 'DJ Mitchell', 'S Dube', 'AM Rahane', 'SN Thakur', 'Mustafizur Rahman', 'DL Chahar', 'M Pathirana', 'M Theekshana', 'TU Deshpande', 'RA Jadeja', 'R Ravindra', 'MM Ali', 'MJ Santner', 'MS Dhoni', 'DP Conway', 'Sameer Rizvi', 'SK Rasheed', 'Mukesh Choudhary', 'RS Hangargekar','Simarjeet Singh', 'PH Solanki', 'N Sindhu', 'AJ Mandal'] #[CSK]

# csv_file = 'E:\Major Project 2\IPL Player Performance Prediction\Players data\output_file.csv'
   
# Read the CSV file into a DataFrame
# final_data = pd.read_csv(csv_file)

total_accuracy = 0
num_models = 0
avg_sum = 0

for player_name in players_list:
    # Filter the data for the current player
    player_data = final_data[final_data['Player'] == player_name]

    # Remove rows with missing values
    player_new = player_data.dropna()

    # Predict next runs
    X_runs = player_new[player_new.columns[2:7]]
    y_runs = player_new[player_new.columns[17:18]]

    X_train, X_test, y_train,y_test = train_test_split(X_runs, y_runs, test_size=0.4, random_state=123)
    ridge_runs = pd.DataFrame()

    # Define the model
    next_runs = Sequential()
    next_runs.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))  # Input layer
    next_runs.add(Dense(16, activation='relu'))  # Hidden layer
    next_runs.add(Dense(1))  # Output layer

    # Compile the model
    next_runs.compile(loss='mean_squared_error', optimizer='RMSprop')

    # Fit the model
    next_runs.fit(X_train, y_train, epochs=20, batch_size=30, verbose=1)
    sd_next_runs = stdev(X_train['Runs Scored'].astype('float'))

    # Get the latest data for the player
    latest = player_new.groupby('Player').tail(1)
    # latest = player.groupby('Player')

    # Predict next runs, balls, overs, runs given, and wickets
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:7]])
    # latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    # latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    # latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    # latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])

    # Calculate confidence intervals for each prediction
    latest['next_runs_ll_95'], latest['next_runs_ul_95'] = latest['next_runs'] - scipy.stats.norm.ppf(.95) * (sd_next_runs / math.sqrt(len(X_train))),latest['next_runs'] + scipy.stats.norm.ppf(.95) * (sd_next_runs / math.sqrt(len(X_train)))
    # latest['next_balls_ll_95'], latest['next_balls_ul_95'] = latest['next_balls'] - scipy.stats.norm.ppf(.95) * (sd_next_balls / math.sqrt(len(X_train_balls))),latest['next_balls'] + scipy.stats.norm.ppf(.95) * (sd_next_balls / math.sqrt(len(X_train_balls)))
    # latest['next_overs_ll_95'], latest['next_overs_ul_95'] = latest['next_overs'] - scipy.stats.norm.ppf(.95) * (sd_next_overs / math.sqrt(len(X_train_overs))), latest['next_overs'] + scipy.stats.norm.ppf(.95) * (sd_next_overs / math.sqrt(len(X_train_overs)))
    # latest['next_runs_given_ll_95'], latest['next_runs_given_ul_95'] = latest['next_runs_given'] - scipy.stats.norm.ppf(.95) * (sd_next_runs_given / math.sqrt(len(X_train_runs_given))), latest['next_runs_given'] + scipy.stats.norm.ppf(.95) * (sd_next_runs_given / math.sqrt(len(X_train_runs_given)))
    # latest['next_wkts_ll_95'], latest['next_wkts_ul_95'] = latest['next_wkts'] - scipy.stats.norm.ppf(.95) * (sd_next_wkts / math.sqrt(len(X_train_wkts))),latest['next_wkts'] + scipy.stats.norm.ppf(.95) * (sd_next_wkts / math.sqrt(len(X_train_wkts)))


    # Append the latest predictions to the models dataframe
    models = models.append(latest)
  


Epoch 1/20
3/3 [==============================] - 1s 4ms/step - loss: 4195.6484
Epoch 2/20
3/3 [==============================] - 0s 4ms/step - loss: 2089.5288
Epoch 3/20
3/3 [==============================] - 0s 4ms/step - loss: 1302.6427
Epoch 4/20
3/3 [==============================] - 0s 4ms/step - loss: 895.2529
Epoch 5/20
3/3 [==============================] - 0s 3ms/step - loss: 667.1081
Epoch 6/20
3/3 [==============================] - 0s 4ms/step - loss: 548.4424
Epoch 7/20
3/3 [==============================] - 0s 4ms/step - loss: 491.9111
Epoch 8/20
3/3 [==============================] - 0s 4ms/step - loss: 467.8865
Epoch 9/20
3/3 [==============================] - 0s 4ms/step - loss: 454.5952
Epoch 10/20
3/3 [==============================] - 0s 4ms/step - loss: 446.7055
Epoch 11/20
3/3 [==============================] - 0s 3ms/step - loss: 446.9456
Epoch 12/20
3/3 [==============================] - 0s 3ms/step - loss: 445.5494
Epoch 13/20
3/3 [=============================

In [17]:
models

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,Bat Dot%,Overs Bowled,Runs Given,Wickets Taken,...,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts,next_runs_ll_95,next_runs_ul_95
116,SA Yadav,smat2021_095,22,15,1,146.7,26.7,0.0,0.0,0.0,...,0.0,0.0,0.0,23.710449,0,0.0,0.0,0.0,19.422445,27.998453
248,RG Sharma,ipl2024_08,18,7,0,257.1,28.6,0.0,0.0,0.0,...,0.0,0.0,0.0,35.426628,0,0.0,0.0,0.0,31.125931,39.727325
283,NT Tilak Varma,smat2023_069,40,24,0,166.7,29.2,0.0,0.0,0.0,...,0.0,0.0,0.0,24.881062,0,0.0,0.0,0.0,16.943542,32.818581
359,TM Head,t20blast2021_108,5,8,1,62.5,37.5,0.0,0.0,0.0,...,0.0,0.0,0.0,15.968111,0,0.0,0.0,0.0,10.560699,21.375523
398,D Brevis,sat20_2024_28,9,10,1,90.0,40.0,0.0,0.0,0.0,...,0.0,0.0,0.0,22.530836,0,0.0,0.0,0.0,16.703360,28.358313
507,JJ Bumrah,ipl2024_08,0,0,0,0.0,0.0,4.0,36.0,0.0,...,2.0,1.0,20.8,0.044008,0,0.0,0.0,0.0,-0.192787,0.280803
525,G Coetzee,sat20_2023_32,12,17,1,70.6,64.7,4.0,41.0,0.0,...,2.0,2.0,37.5,-6.571690,0,0.0,0.0,0.0,-8.520993,-4.622386
618,PP Chawla,smat2023_127,13,8,0,162.5,12.5,4.0,19.0,0.0,...,1.0,0.0,37.5,-7.962517,0,0.0,0.0,0.0,-9.185746,-6.739288
653,N Thushara,sat20_2024_28,0,1,1,0.0,100.0,3.4,40.0,1.0,...,6.0,0.0,40.9,-0.049759,0,0.0,0.0,0.0,-0.622667,0.523149
784,L Wood,t20blast2023_127,0,0,0,0.0,0.0,4.0,37.0,1.0,...,3.0,1.0,50.0,0.052081,0,0.0,0.0,0.0,-1.121451,1.225614
